In [1]:
# Analyzing the mineral projects in Argentina

# import the needed libraries
import json

import geopandas as gpd
import pandas as pd

# Read the data into a pandas.DataFrame
dataset_url = "http://datos.energia.gob.ar/dataset/05ceffb0-98f3-462b-98f9-96d655f76cef/resource/0c8317ac-eacb-4826-b7e7-8f0a15a62ac1/download/proyectos-mineros-ubicacin-aproximada-.csv"
# dataset_url_shp = "http://datos.energia.gob.ar/dataset/05ceffb0-98f3-462b-98f9-96d655f76cef/resource/14ae10ff-8750-4019-b837-ba34778f8d23/download/proyectos-mineros-ubicacin-aproximada-.zip"

df_mining_projects = pd.read_csv(dataset_url)
# gdf_mining_projects = gpd.read_file(dataset_url_shp)

print(f"There are {df_mining_projects.shape[0]} mining projects in Argentina")

print("The original dataset:")
print()
df_mining_projects.head()

There are 82 mining projects in Argentina
The original dataset:



,geom,nombre,empresa,tipo_yacim,estado,mineral_pr,geojson
0,0104000020E6100000010000000101000000713D0AD7A3...,Cauchari-Olaroz,MINERA EXAR S.A.,NaN,Construcción,Litio,"{""type"":""MultiPoint"",""coordinates"":[[-66.76,-2..."
1,0104000020E6100000010000000101000000D7A3703D0A...,Bajo La Alumbrera,MINERA ALUMBRERA LIMITED,NaN,Mantenimiento,Cobre,"{""type"":""MultiPoint"",""coordinates"":[[-66.61,-2..."
2,0104000020E6100000010000000101000000448B6CE7FB...,Río Grande,LITHEA INC. SUCURSAL ARGENTINA,NaN,Exploración avanzada,Litio,"{""type"":""MultiPoint"",""coordinates"":[[-68.156,-..."
3,0104000020E610000001000000010100000085EB51B81E...,Navidad,MINERA ARGENTA S.A.,NaN,Evaluación económica preliminar,Plata,"{""type"":""MultiPoint"",""coordinates"":[[-68.83,-4..."
4,0104000020E61000000100000001010000006666666666...,Fenix,MINERA DEL ALTIPLANO S.A.,NaN,Producción,Litio,"{""type"":""MultiPoint"",""coordinates"":[[-67.1,-25..."


Before applying some statistic to the dataset, we can extract the location (longitude, latitude)

Then we can construct a GeoDataFrame and add some information related with the province

In [ ]:
#Extract the coordinates of each mining project in Argentina

# First, apply json.loads function to convert the JSON string in the geojson field into a dictionary
df_mining_projects["geojson"] = df_mining_projects["geojson"].apply(json.loads)

# Second, create a new column for containing the geometry
# Apply an anonimous function (lambda) to the geojson column to extract the coordinates
df_mining_projects["lon"] = df_mining_projects["geojson"].apply(lambda dic: dic["coordinates"][0][0])
df_mining_projects["lat"] = df_mining_projects["geojson"].apply(lambda dic: dic["coordinates"][0][1])

df_mining_projects.drop(columns=["geojson"], inplace=True)

df_mining_projects.head()

In [ ]:
# Construct a geopandas.GeoDataFrame

gdf_mining_projects = gpd.GeoDataFrame(df_mining_projects, geometry=gpd.points_from_xy(df_mining_projects["lon"], df_mining_projects["lat"]), crs=4326)

gdf_mining_projects.head()

In [ ]:
# Let's add information corresponding to the province

gdf_province_arg = gpd.read_file("files/provincia.json", usecols=["nam", "geometry"])

gdf_province_arg.head()

In [ ]:
gdf_mining_projects = gdf_mining_projects.sjoin(gdf_province_arg, how="left")

gdf_mining_projects.head()